In [1]:
from time import sleep
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import os
from selenium.webdriver.support.ui import WebDriverWait
import requests
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.common.exceptions import TimeoutException
import datetime
from datetime import datetime, timedelta
from tqdm import tqdm

In [2]:
def scrape_kayak(from_location, to_location, date):

    formatted_date = date.strftime('%Y-%m-%d')
    url = f'https://www.kayak.com/flights/{from_location}-{to_location},nearby/{formatted_date}?sort=bestflight_a'
    driver.get(url)

    
    wait_time = 20  # increase wait time if needed
    wait = WebDriverWait(driver, wait_time)

    while True:
        try:
            show_more_buttons = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.ULvh-button.show-more-button')))
            num_buttons = len(show_more_buttons)

            for i in range(num_buttons):
                show_more_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.ULvh-button.show-more-button')))
                driver.execute_script("arguments[0].click();", show_more_button)
                try:
                    wait.until(EC.staleness_of(show_more_button))
                except:
                    pass

            flight_rows = driver.find_elements(By.XPATH, '//div[@class="nrc6-inner"]')

        except TimeoutException:
            break
    
    return flight_rows

In [3]:
def convert_to_df(flight_rows):
    
    list_prices = []
    list_company_names=[]
    list_stops=[]
    list_duration=[]
    list_dates=[]
    
    for WebElement in flight_rows:
        elementHTML = WebElement.get_attribute('outerHTML')
        elementSoup = BeautifulSoup(elementHTML, 'html.parser')
        # price
        temp_price = elementSoup.find("div", {"class": "nrc6-price-section"})
        price = temp_price.find("div", {"class": "f8F1-price-text"})
        list_prices.append(price.text)

        #company names
        company_names = elementSoup.find("div",{"class":"c_cgF c_cgF-mod-variant-default"}).text
        list_company_names.append(company_names)

        # stops
        temp_stops = elementSoup.find("div", {"class": "vmXl vmXl-mod-variant-default"})
        stops = temp_stops.find("span", {"class": "JWEO-stops-text"})
        list_stops.append(stops.text)

        #durations  
        temp_durations = elementSoup.find("div", {"class": "xdW8"})
        durations = temp_durations.find("div", {"class": "vmXl vmXl-mod-variant-default"}).text
        list_duration.append(durations)
    
    df = pd.DataFrame({
        'Price': list_prices,
        'Company Name': list_company_names,
        'Stops': list_stops,
        'Duration': list_duration,
        })

    df['Destination'] = to_location
    df['From'] = from_location
    df['Date'] = date
    
    return df

In [211]:
driver = webdriver.Chrome()

from_location = 'LAX'
to_location = 'ATL'

date = datetime.strptime('2023-07-05', '%Y-%m-%d')
end_date = datetime.strptime('2023-07-05', '%Y-%m-%d')
total_days = (end_date - date).days + 1

df = pd.DataFrame()
with tqdm(total=total_days) as pbar:

    while date <= end_date:

        df = pd.concat([df, convert_to_df(scrape_kayak(from_location, to_location, date))])
        
        date += timedelta(days=1)
        pbar.update(1)

100%|█████████████████████████████████████████████| 1/1 [00:24<00:00, 24.66s/it]


In [213]:
df

,Price,Company Name,Stops,Duration,Destination,From,Date
0,$139,Spirit Airlines,nonstop,4h 32m,ATL,LAX,2023-07-05
1,$139,Spirit Airlines,nonstop,4h 34m,ATL,LAX,2023-07-05
2,$162,Spirit Airlines,1 stop,7h 48m,ATL,LAX,2023-07-05
3,$162,Spirit Airlines,1 stop,7h 51m,ATL,LAX,2023-07-05
4,$162,Spirit Airlines,1 stop,8h 35m,ATL,LAX,2023-07-05
5,$162,Spirit Airlines,1 stop,13h 47m,ATL,LAX,2023-07-05
6,$174,Spirit Airlines,1 stop,16h 40m,ATL,LAX,2023-07-05
7,$174,Spirit Airlines,1 stop,18h 06m,ATL,LAX,2023-07-05
8,$174,Spirit Airlines,1 stop,20h 59m,ATL,LAX,2023-07-05
9,$191,Spirit Airlines,1 stop,8h 18m,ATL,LAX,2023-07-05


In [214]:
df.to_csv('flight_LAX_ATL_0705_data.csv', index=False)

In [225]:
# Read the individual CSV files into separate dataframes
df1 = pd.read_csv('flight_LAX_ATL_0801_data.csv')
df2 = pd.read_csv('flight_LAX_ATL_0802_data.csv')
df3 = pd.read_csv('flight_LAX_ATL_0803_data.csv')
df4 = pd.read_csv('flight_LAX_ATL_0804_data.csv')
df5 = pd.read_csv('flight_LAX_ATL_0805_data.csv')
df6 = pd.read_csv('flight_LAX_ATL_0806_data.csv')
df7 = pd.read_csv('flight_LAX_ATL_0807_data.csv')
df8 = pd.read_csv('flight_LAX_ATL_0808_data.csv')
df9 = pd.read_csv('flight_LAX_ATL_0809_data.csv')
df10 = pd.read_csv('flight_LAX_ATL_0810_data.csv')
df11 = pd.read_csv('flight_LAX_ATL_0811_data.csv')
df12 = pd.read_csv('flight_LAX_ATL_0812_data.csv')
df13 = pd.read_csv('flight_LAX_ATL_0813_data.csv')
df14 = pd.read_csv('flight_LAX_ATL_0814_data.csv')
df15 = pd.read_csv('flight_LAX_ATL_0815_data.csv')
df16 = pd.read_csv('flight_LAX_ATL_0816_data.csv')
df17 = pd.read_csv('flight_LAX_ATL_0817_data.csv')
df18 = pd.read_csv('flight_LAX_ATL_0818_data.csv')
df19 = pd.read_csv('flight_LAX_ATL_0819_data.csv')
df20 = pd.read_csv('flight_LAX_ATL_0820_data.csv')
df21 = pd.read_csv('flight_LAX_ATL_0821_data.csv')
df22 = pd.read_csv('flight_LAX_ATL_0822_data.csv')
df23 = pd.read_csv('flight_LAX_ATL_0823_data.csv')
df24 = pd.read_csv('flight_LAX_ATL_0824_data.csv')
df25 = pd.read_csv('flight_LAX_ATL_0825_data.csv')
df26 = pd.read_csv('flight_LAX_ATL_0826_data.csv')
df27 = pd.read_csv('flight_LAX_ATL_0827_data.csv')
df28 = pd.read_csv('flight_LAX_ATL_0828_data.csv')
df29 = pd.read_csv('flight_LAX_ATL_0829_data.csv')
df30 = pd.read_csv('flight_LAX_ATL_0830_data.csv')
df31 = pd.read_csv('flight_LAX_ATL_0831_data.csv')
df32 = pd.read_csv('flight_LAX_ATL_0701_data.csv')
df33 = pd.read_csv('flight_LAX_ATL_0702_data.csv')
df34 = pd.read_csv('flight_LAX_ATL_0703_data.csv')
df35 = pd.read_csv('flight_LAX_ATL_0704_data.csv')
df36 = pd.read_csv('flight_LAX_ATL_0705_data.csv')
df37 = pd.read_csv('flight_LAX_ATL_0706_data.csv')
df38 = pd.read_csv('flight_LAX_ATL_0707_data.csv')
df39 = pd.read_csv('flight_LAX_ATL_0708_data.csv')
df40 = pd.read_csv('flight_LAX_ATL_0709_data.csv')
df41 = pd.read_csv('flight_LAX_ATL_0710_data.csv')
df42 = pd.read_csv('flight_LAX_ATL_0711_data.csv')
df43 = pd.read_csv('flight_LAX_ATL_0712_data.csv')
df44 = pd.read_csv('flight_LAX_ATL_0713_data.csv')
df45 = pd.read_csv('flight_LAX_ATL_0714_data.csv')
df46 = pd.read_csv('flight_LAX_ATL_0715_data.csv')
df47 = pd.read_csv('flight_LAX_ATL_0716_data.csv')
df48 = pd.read_csv('flight_LAX_ATL_0717_data.csv')
df49 = pd.read_csv('flight_LAX_ATL_0718_data.csv')
df50 = pd.read_csv('flight_LAX_ATL_0719_data.csv')
df51 = pd.read_csv('flight_LAX_ATL_0720_data.csv')
df52 = pd.read_csv('flight_LAX_ATL_0721_data.csv')
df53 = pd.read_csv('flight_LAX_ATL_0722_data.csv')
df54 = pd.read_csv('flight_LAX_ATL_0723_data.csv')
df55 = pd.read_csv('flight_LAX_ATL_0724_data.csv')
df56 = pd.read_csv('flight_LAX_ATL_0725_data.csv')
df57 = pd.read_csv('flight_LAX_ATL_0726_data.csv')
df58 = pd.read_csv('flight_LAX_ATL_0727_data.csv')
df59 = pd.read_csv('flight_LAX_ATL_0728_data.csv')
df60 = pd.read_csv('flight_LAX_ATL_0729_data.csv')
df61 = pd.read_csv('flight_LAX_ATL_0730_data.csv')
df62 = pd.read_csv('flight_LAX_ATL_0731_data.csv')


# Concatenate the dataframes into a single dataframe
combined_df = pd.concat([df32, df33, df34,df35,df36,
                        df37,df38,df39,df40,df41,df42,df43,df44,df45,df46,df47,
                        df48,df49,df50,df51,df52,df53,df54,df55,df56,df57,
                        df58,df59,df60,df61,df62, df1, df2, df3,df4,df5,df6,df7,df8,df9,df10,
                        df11,df12,df13,df14,df15,df16,df17,df18,df19,
                        df20, df21, df22, df23, df24, df25, df26, 
                         df27, df28, df29, df30, df31], ignore_index=True)

# Write the combined dataframe to a new CSV file
combined_df.to_csv('flight_LAX_ATL_combined_0701_0831_data.csv', index=False)


In [226]:
combined_df

,Price,Company Name,Stops,Duration,Destination,From,Date
0,$393,United Airlines,1 stop,8h 18m,ATL,LAX,2023-07-01
1,$679,Delta,nonstop,4h 29m,ATL,LAX,2023-07-01
2,$252,Spirit Airlines,1 stop,8h 52m,ATL,LAX,2023-07-01
3,$509,American Airlines,1 stop,7h 07m,ATL,LAX,2023-07-01
4,$344,American Airlines,1 stop,9h 05m,ATL,LAX,2023-07-01
...,...,...,...,...,...,...,...
26521,Info,Southwest,1 stop,6h 55m,ATL,LAX,2023-08-31
26522,Info,Southwest,1 stop,7h 00m,ATL,LAX,2023-08-31
26523,Info,Southwest,1 stop,7h 10m,ATL,LAX,2023-08-31
26524,Info,Southwest,1 stop,7h 10m,ATL,LAX,2023-08-31


In [255]:
combined_df2 = combined_df[combined_df['Price'] != 'Info'].reset_index(drop=True)
combined_df2

,Price,Company Name,Stops,Duration,Destination,From,Date
0,$393,United Airlines,1 stop,8h 18m,ATL,LAX,2023-07-01
1,$679,Delta,nonstop,4h 29m,ATL,LAX,2023-07-01
2,$252,Spirit Airlines,1 stop,8h 52m,ATL,LAX,2023-07-01
3,$509,American Airlines,1 stop,7h 07m,ATL,LAX,2023-07-01
4,$344,American Airlines,1 stop,9h 05m,ATL,LAX,2023-07-01
...,...,...,...,...,...,...,...
26228,"$1,174","Avianca, JetBlue",3 stops,32h 56m,ATL,LAX,2023-08-31
26229,"$1,174","Avianca, JetBlue",3 stops,37h 39m,ATL,LAX,2023-08-31
26230,"$1,201","Avianca, JetBlue",3 stops,37h 39m,ATL,LAX,2023-08-31
26231,"$1,174","Avianca, JetBlue",3 stops,38h 34m,ATL,LAX,2023-08-31


In [256]:
combined_df2.to_csv('flight_LAX_ATL_combined_0701_0831_clean_data.csv', index=False)

# ALSO NEED TO CLEAN THE "" FOR PRICE AND COMPANY NAME